In [2]:
import tensorflow as tf


In [3]:
v=tf.Variable(0.0)
(v+1).numpy()

1.0

In [4]:
v.assign(5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=5.0>

In [5]:
v.assign_add(1)
v.read_value()

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [6]:
w=tf.Variable([[1.0]])
with tf.GradientTape() as t: #自动跟踪变量的梯度计算
    loss = w*w


In [7]:
grad=t.gradient(loss,w)
grad

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [8]:
d=tf.constant([[3.0]])
with tf.GradientTape() as t: #自动跟踪变量的梯度计算
    t.watch(d) #常量要用watch
    dloss = d*d

In [9]:
dloss_d = t.gradient(dloss,d)
dloss_d

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[6.]], dtype=float32)>

In [10]:
d=tf.constant([[3.0]])
with tf.GradientTape(persistent=True) as t: #持久
    t.watch(d) #常量要用watch
    y = d*d
    z = y*y

dy_d = t.gradient(y,d)
dy_d

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[6.]], dtype=float32)>

In [11]:
dz_d = t.gradient(z,d)
dz_d

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[108.]], dtype=float32)>

In [12]:
(train_image,train_labels),(test_image,test_labels)=tf.keras.datasets.mnist.load_data()

In [13]:
train_image=tf.expand_dims(train_image,-1)

test_image=tf.expand_dims(test_image,-1)

In [14]:
train_image.shape

TensorShape([60000, 28, 28, 1])

In [15]:
train_image = tf.cast(train_image/255,tf.float32)
test_image = tf.cast(test_image/255,tf.float32)

In [16]:
train_labels = tf.cast(train_labels,tf.int64)
test_labels = tf.cast(test_labels,tf.int64)

In [17]:
dataset = tf.data.Dataset.from_tensor_slices((train_image,train_labels))
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [18]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_image,test_labels))

In [19]:
dataset = dataset.shuffle(10000).batch(32).repeat(1)
dataset

<RepeatDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [20]:
test_dataset=test_dataset.batch(32)

In [21]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,[3,3],activation='relu',input_shape=(None,None,1)),
    tf.keras.layers.Conv2D(32,[3,3],activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10)
])

In [22]:
optimizer=tf.keras.optimizers.Adam()

In [23]:
loss_fun=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

标签为顺序编码

sparse_categorical_crossentropy() 需要指定y_true和y_pred

SparseCategoricalCrossentropy() 直接调用，不需要指定y_true和y_pred

标签为独热编码

categorical_crossentropy()


In [24]:
feature,labels = next(iter(dataset))
feature.shape

TensorShape([32, 28, 28, 1])

In [25]:
predictions = model(feature)
predictions.shape

TensorShape([32, 10])

In [26]:
tf.argmax(predictions,axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([2, 2, 2, 2, 2, 2, 6, 2, 6, 2, 2, 2, 2, 6, 6, 6, 2, 2, 6, 2, 2, 2,
       2, 6, 2, 6, 2, 6, 2, 6, 2, 2], dtype=int64)>

In [27]:
labels

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([8, 0, 2, 2, 0, 7, 2, 1, 9, 1, 4, 1, 8, 3, 7, 3, 5, 8, 3, 3, 9, 0,
       0, 7, 4, 9, 0, 7, 9, 7, 6, 9], dtype=int64)>

In [28]:
def loss(model,x,y):
    y_ = model(x)
    return loss_fun(y,y_)

In [29]:
train_loss=tf.keras.metrics.Mean('train_loss')
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss=tf.keras.metrics.Mean('test_loss')
test_accuracy=tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [30]:
def train_step(model,images,labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step=loss_fun(labels,pred)
        #loss_step = loss(model,images,labels)

    grads = t.gradient(loss_step,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    train_loss(loss_step)
    train_accuracy(labels,pred)

In [31]:
def test_step(model,images,labels):
    pred = model(images)
    loss_step=loss_fun(labels,pred)
    test_loss(loss_step)
    test_accuracy(labels,pred)

In [32]:
def train():
    for epoch in range(10):
        for (batch,(images,labels)) in enumerate(dataset):
            train_step(model,images,labels)
        print("Epoch{} train_loss is {}, train_accuracy is {}".format(epoch,
                                                          train_loss.result(),
                                                          train_accuracy.result()))
        for (batch,(images,labels)) in enumerate(test_dataset):
            test_step(model,images,labels)
        print("Epoch{} test_loss is {}, test_accuracy is {}".format(epoch,
                                                          test_loss.result(),
                                                          test_accuracy.result()))
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()




In [33]:
train()

Epoch0 train_loss is 1.7059605121612549, train_accuracy is 0.3719500005245209
Epoch0 test_loss is 1.4552584886550903, test_accuracy is 0.4440000057220459
Epoch1 train_loss is 1.3196732997894287, train_accuracy is 0.5436499714851379
Epoch1 test_loss is 1.1206575632095337, test_accuracy is 0.612500011920929
Epoch2 train_loss is 1.0488978624343872, train_accuracy is 0.6539833545684814
Epoch2 test_loss is 0.9080567955970764, test_accuracy is 0.7059000134468079
Epoch3 train_loss is 0.8866193294525146, train_accuracy is 0.7141666412353516
Epoch3 test_loss is 0.7660938501358032, test_accuracy is 0.7682999968528748
Epoch4 train_loss is 0.781707227230072, train_accuracy is 0.7551500201225281
Epoch4 test_loss is 0.6887974143028259, test_accuracy is 0.7879999876022339
Epoch5 train_loss is 0.7051310539245605, train_accuracy is 0.7801833152770996
Epoch5 test_loss is 0.639205813407898, test_accuracy is 0.7986000180244446
Epoch6 train_loss is 0.6446504592895508, train_accuracy is 0.801716685295105
Ep

In [34]:
# m = tf.keras.metrics.Mean('acc')
# m(10)
#

In [35]:
# m(20)
#

In [36]:
# m.result().numpy()
#

In [37]:
# m([30,40])
#

In [38]:
# m.result()
#

In [39]:
# m.reset_states()
# m(1)
#

In [40]:
# m(2)
# m.result().numpy()
#

In [41]:
# a = tf.keras.metrics.SparseCategoricalAccuracy('acc')
# a(labels,model(feature))
